In [97]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from helperClasses.DataPipeline import DataPipeline
from helperClasses.BaseTransformation import BaseTransformation

In [98]:
df = pd.read_csv("../data/df_detected_objects_game.csv")
df = df.drop(columns= ["head_x", "head_y", "head_z"])

# Hilfe Funktionen

In [99]:
P1=np.array([-4.207, -1.846, -5.733])
P2=np.array([3.415, -1.870, -5.489])
P3=np.array([-0.057, -1.703, -15.554])

In [100]:
def Tennisfeld(r,P1,P2,P3):
    r1=P2-P1
    r2=-(P3-0.5*(P1+P2))
    # Basisvektoren
    r1u=r1/np.sqrt(np.dot(r1,r1))
    r2u=r2/np.sqrt(np.dot(r2,r2))
    r3u=np.cross(r1u,r2u)
    # Translation
    T0=0.5*(P1+P2)-r2u
    # Rotation
    M1=np.c_[r1u,r2u,r3u]
    M1_inv=np.linalg.inv(M1)
    # Abbildung
    return np.inner(M1_inv,(r-T0))  

In [101]:
Tennisfeld(P1,P1,P2,P3)

array([-3.81297115e+00,  1.00000000e+00, -5.53413430e-17])

df["x"] = df["x"].apply(lambda x: Tennisfeld(np.array([x,0,0]),P1,P2,P3)[0])
df["y"] = df["y"].apply(lambda x: Tennisfeld(np.array([0,x,0]),P1,P2,P3)[1])
df["z"] = df["z"].apply(lambda x: Tennisfeld(np.array([0,0,x]),P1,P2,P3)[2])

calibration_points = np.array([
    [-4.207, -1.846, -5.733], # service line center
    [3.415, -1.870, -5.489], # service x singles sideline
    [-0.057, -1.703, -15.554] # baseline x singles sideline
])
bt = BaseTransformation(calibration_points)
df = bt.transformData(df, [-1.5, 0, -18.28])

In [106]:
def drawTennisField():
    plt.plot([4.11, 4.11], [-11.89, 11.89], 'k', lw=2) # Single sideline rechts
    plt.plot([5.48, 5.48], [-11.89, 11.89], 'k', lw=2) # Double sideline rechts
    plt.plot([-4.11, -4.11], [-11.89, 11.89], 'k', lw=2) # Single sideline links
    plt.plot([-5.48, -5.48], [-11.89, 11.89], 'k', lw=2) # Double sideline links
    plt.plot([0, 0], [-6.40, 6.40], 'k', lw=2) # Center Service Line
    plt.plot([-5.48, 5.48], [0, 0], 'k', lw=2) # Net
    plt.plot([-5.48, 5.48], [11.89, 11.89], 'k', lw=2) # Baseline oben
    plt.plot([-5.48, 5.48], [-11.89, -11.89], 'k', lw=2) # Baseline unten
    plt.plot([-4.11, 4.11], [-6.40, -6.40], 'k', lw=2) # Service Line unten
    plt.plot([-4.11, 4.11], [6.40, 6.40], 'k', lw=2) # Service Line oben

# Analyse Tennisspiel

## Datawrangling

In [105]:
df.head()

,frame,object_id,object_label,confidence,tracking_state,x,y,z,vx,xy,vz,width,height,length
0,3,0,Person,79.83398,OK,1.20102,0.41904,1.55682,-1.75535,-0.26043,0.55864,0.49680,0.57462,0.49680
1,4,0,Person,87.45117,OK,1.15833,0.41409,1.56276,-1.56742,-0.20237,0.39540,0.49763,0.58033,0.49763
2,5,0,Person,89.20898,OK,1.11960,0.40943,1.56608,-1.45386,-0.16676,0.29555,0.50174,0.58424,0.50174
3,6,0,Person,90.03906,OK,1.08496,0.40524,1.56808,-1.35545,-0.14275,0.23192,0.51613,0.58824,0.51613
4,10,2,Person,91.06445,OK,4.09950,1.25795,5.92474,-4.40850,0.16304,-1.38635,2.54048,2.63067,2.54048


object_id scheint zu viele unterschiedliche labels zu haben